In [1]:
import numba
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import convolve, generate_binary_structure
import matplotlib
matplotlib.use('QT5Agg')
%matplotlib qt

In [99]:
class isingmodel:
    def __init__(self, N, J=1):
        self.J = J
        self.N = N
        self.init_random = np.random.random((N, N))
        self.grid_n = np.ones((N, N))
        # self.grid_n[self.init_random >= .5] = 1
        # self.grid_n[self.init_random < .5] = -1
    
    def get_energy(self, grid):
        kernel = generate_binary_structure(2, 1)
        kernel[1][1] = False
        result = - self.J * grid * convolve(grid, kernel, mode='wrap')
        return result.sum()
    
    def grid(self):
        return self.grid_n
    
    @staticmethod
    @numba.njit("Tuple((f8[:], f8[:], f8[:,:], f8[:]))(f8[:,:], i8, f8, f8, i8)", nopython=True, nogil=True)
    def metropolis(arr_spin, times, beta, energy, N):
        arr_spin = arr_spin.copy()
        total_spin = np.zeros(times - 1)
        total_energy = np.zeros(times - 1)
        # norm_variance = np.zeros(times -1)
        for t in range(0, times - 1):
            x = np.random.randint(0,N)
            y = np.random.randint(0,N)
            spin_t = arr_spin[x, y]
            spin_prime = -spin_t
            E_t = 0
            E_prime = 0
            neighbours = [(x - 1) % N, (x + 1) % N, (y - 1) % N, (y + 1) % N]
            for nx in [neighbours[0], neighbours[1]]:
                E_t += - spin_t * arr_spin[nx, y]
                E_prime += - spin_prime * arr_spin[nx, y]
            for ny in [neighbours[2], neighbours[3]]:
                E_t += - spin_t * arr_spin[x, ny]
                E_prime += - spin_prime * arr_spin[x, ny]
            dE = E_prime - E_t
            if (dE > 0) & (np.random.random() < np.exp(-beta * dE)):
                arr_spin[x, y] = spin_prime
                energy += dE
            elif dE <= 0:
                arr_spin[x, y] = spin_prime
                energy += dE
            total_spin[t] = arr_spin.sum()
            # norm_variance[t] = arr_spin.var()
            total_energy[t] = energy
        
        mag_squared = total_spin ** 2
        
        
        return total_spin, total_energy, arr_spin, mag_squared
        
    def plot(self, cmap, times, beta, save=False):
        fig, axes = plt.subplots(dpi=150)
        plt.rcParams["font.family"] = "times"
        plt.rcParams["text.usetex"] = True
        plt.title(fr'{self.N}$\times${self.N}-lattice after {times:.1e} thermalization steps' + '\n' + rf'$\beta={beta}$', fontsize=12, pad=10)
        plt.imshow(equilibrium, cmap=cmap)
        if save:
            fig.savefig(f"/Users/danielmiksch/Downloads/{self.N}by{self.N}_grid.pdf")
            
    def magnetization(self, spins):
        return spins / self.N ** 2
    
    def print_energy(self):
        print(self.get_energy(self.grid_n))

/Users/danielmiksch/miniconda3/envs/juplab/lib/python3.10/site-packages/numba/core/decorators.py:282: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)


# a)
$50\times 50$-grid after $10\,000\cdot 50^2$ thermalization updates at $\beta=0.45$. The lattice is being initialized to the ferromagnetic ground state, i.e. all spins are equal to one.

In [3]:
N = 50
steps = 25e6
beta = 0.45
model = isingmodel(N=N)
spin_grid = model.grid()
energy = model.get_energy(spin_grid)

In [4]:
spins, energies, equilibrium, variance = model.metropolis(spin_grid, steps, beta, energy, N)

In [5]:
model.plot(cmap='binary', times=steps, beta=beta, save=False)

In [6]:
spins[-25000:].mean()

2040.40544

# c)
The following code calculates the magnetisation with the metropolis algorithm for $N=20$ and $N=50$ on an interval of $\beta\in [0.3,0.7]$. To avoid fluctuation of the magnetization, the last $500\cdot N^2$ measurements are averaged.
The results of this computation are shown in the figure below. As one can see, the magnetization curve of the $50\times 50$-lattice are much smoother than the one of the $20\times 20$-lattice.
One reason for this is the size of the selected system. In most thermodynamic systems, the assumption is made that the system is infinite. A larger lattice has more spins that can interact. This leads to a larger number of microstates that determine the macro behavior of the system.
This results in fewer statistical fluctuations of the observables and thus a smoother magnetization curve.

In [39]:
betas = np.linspace(0.3, 0.7, 20)
N = [20, 50]
steps = [10000 * i**2 for i in N]
sweeps = [500 * i**2 for i in N]

In [40]:
def get_magnetization(N, betas, sweeps):
    magnetization_list = np.zeros((len(N), len(betas)))
    for index1, i in enumerate(N):
        magnetization = np.zeros(len(betas))
        model = isingmodel(N=i)
        spin_grid = model.grid()
        energy = model.get_energy(spin_grid)
        for index2, s in enumerate(betas):
            spins, energies, equilibrium, variance = model.metropolis(spin_grid, steps[index1], s, energy, i)
            magnetization[index2] = spins[-sweeps[index1]:].mean() / i**2
        magnetization_list[index1] = magnetization
            
    return magnetization_list

In [41]:
magnetization = get_magnetization(N, betas, sweeps)
magnetization

array([[ 0.0109261 ,  0.001713  ,  0.00420068, -0.04097008, -0.1634911 ,
         0.4256311 ,  0.54817665,  0.79817728,  0.856505  ,  0.87797697,
         0.92154005,  0.93979255,  0.95728353,  0.96647735,  0.97422165,
         0.97798942,  0.97998155,  0.9856493 ,  0.98785025,  0.9893464 ],
       [-0.00351969, -0.02724276, -0.01629092, -0.01553097, -0.01239974,
         0.03463546,  0.21955268,  0.73917898,  0.86186742,  0.88989431,
         0.92393705,  0.94228004,  0.95622893,  0.96579345,  0.97221121,
         0.97787851,  0.98204236,  0.98480609,  0.98772919,  0.99026219]])

In [55]:
fig, ax = plt.subplots()

ax.plot(1 / betas, magnetization[0], 'o--', c='blue', label=r'$20\times20$-lattice')
ax.plot(1 / betas, magnetization[1], 'o--', c='red', label=r'$50\times50$-lattice')
ax.axhline(0, c='grey', linewidth=.8)
ax.set_xlabel(r'$1/\beta$')
ax.set_ylabel(r'Magnetization $\langle M\rangle$')
plt.title('Magnetization of different lattices', fontsize=12, pad=10)

legend = ax.legend(loc="upper right", bbox_to_anchor=(0.92, 0.92), fancybox=False, edgecolor='black', fontsize=12)
legend.set_zorder(10)
legend.get_frame().set_linewidth(0.5)

plt.show()

In [ ]:
# fig, axes = plt.subplots(1, 2, figsize=(12,4))
# 
# plt.rcParams["font.family"] = "times"
# plt.rcParams["text.usetex"] = True
# 
# ax = axes[0]
# ax.plot(spins/50**2)
# ax.set_xlabel('Algorithm Time Steps')
# ax.set_ylabel(r'Magnetization $\langle M\rangle$')
# ax.grid()
# ax = axes[1]
# ax.plot(energies)
# ax.set_xlabel('Algorithm Time Steps')
# ax.set_ylabel(r'Energy $E$')
# ax.grid()
# fig.tight_layout()
# fig.suptitle(r'Evolution of Average Spin and Energy', y=1.07, size=18)
# plt.show()

# d)

In [126]:
betas = np.linspace(0.3, 0.7, 20)
N = [20, 50]
steps = [10000 * i ** 2 for i in N]
sweeps = [600 * i ** 2 for i in N]

In [127]:
def get_magnetization(N, betas, sweeps):
    variance_list = np.zeros((len(N), len(betas)))
    for index1, i in enumerate(N):
        variance = np.zeros(len(betas))
        model = isingmodel(N=i)
        spin_grid = model.grid()
        energy = model.get_energy(spin_grid)
        for index2, s in enumerate(betas):
            spins, energies, equilibrium, mag_squared = model.metropolis(spin_grid, steps[index1], s, energy, i)
            variance[index2] = beta * (mag_squared[-sweeps[index1]:].mean() - spins[-sweeps[index1]:].mean() ** 2) / i**2
        variance_list[index1] = variance

    return variance_list

In [128]:
variance = get_magnetization(N, betas, sweeps)

In [129]:
variance

array([[4.23637318e+00, 6.48776817e+00, 9.21784376e+00, 2.02425153e+01,
        1.92234617e+01, 1.87984785e+01, 7.40000379e+00, 5.29998396e+00,
        1.17735125e+00, 1.44203712e+00, 2.42426470e-01, 2.02178901e-01,
        1.26060607e-01, 1.23729902e-01, 5.06868538e-02, 6.12197150e-02,
        4.47886637e-02, 3.49408299e-02, 1.94462317e-02, 1.69449796e-02],
       [5.47339810e+00, 6.80279635e+00, 8.78538560e+00, 1.09256116e+01,
        1.41011402e+01, 4.31103823e+01, 2.08770825e+02, 1.46301852e+01,
        2.36193879e+00, 6.90456768e-01, 2.90349619e-01, 2.52328388e-01,
        1.63122408e-01, 1.13570352e-01, 6.45294475e-02, 4.80211902e-02,
        4.15942776e-02, 2.95113955e-02, 2.49789672e-02, 1.99977009e-02]])

In [130]:
fig, ax = plt.subplots()
ax.plot(1 / betas, variance[0], 'o--', c='blue', label=r'$20\times20$-lattice')
ax.plot(1 / betas, variance[1], 'o--', c='red', label=r'$50\times50$-lattice')

In [76]:
# fig, axes = plt.subplots(1, 2, figsize=(12,4))
# 
# plt.rcParams["font.family"] = "times"
# plt.rcParams["text.usetex"] = True
# 
# ax = axes[0]
# ax.plot(variance)
# ax.set_xlabel('Algorithm Time Steps')
# ax.set_ylabel(r'Magnetization $\langle M\rangle$')
# ax.grid()
# ax = axes[1]
# ax.plot(energies)
# ax.set_xlabel('Algorithm Time Steps')
# ax.set_ylabel(r'Energy $E$')
# ax.grid()
# fig.tight_layout()
# fig.suptitle(r'Evolution of Average Spin and Energy', y=1.07, size=18)
# plt.show()